In [ ]:
from scipy.integrate import quad_vec
from scipy.interpolate import interp1d
import numpy as np
import pandas as pd
import os
CurrentPath = os.getcwd()
sndata = pd.read_csv(CurrentPath + '/Data/parsonage.txt', sep = ' ', engine='python')

In [45]:
def E_inverse_curved(z, Omega_m, Omega_L): # return 1/E(z) = H0/H(z)
    Omegak = 1 - Omega_m - Omega_L
    E2 = Omega_m*(1+z)**3 + Omega_L + Omegak*(1+z)**2
    E = np.sqrt(E2) 
    return 1/E

def Other_stuff_curved(z, parm): # parm[0] = Omegam, parm[1] = Omegalamb, d_p = c/H0*Other_stuff_flat
    Omegam = parm[0]
    Omega_L = parm[1]
    Omegak = 1 - Omegam - Omega_L
    grid = np.linspace(z.min(), z.max(), 100)
    gridval = np.array([quad_vec(E_inverse_curved, 0,n, args=(Omegam, Omega_L))[0] for n in grid])
    if np.abs(Omegak) < 1e-14: # flat universe
        grid_dist = gridval
    elif Omegak > 1e-14: # open universe
        grid_dist = 1/np.sqrt(Omegak)*np.sinh(np.sqrt(Omegak)*gridval)
    else: # closed universe
        grid_dist = 1/np.sqrt(-Omegak)*np.sin(np.sqrt(-Omegak)*gridval)
    interp_func = interp1d(grid, grid_dist, kind='linear', fill_value='extrapolate')
    integral = interp_func(z)
    return integral
def B(func, parm,z):
    """
    B(Omegam, Omegalamb) = 5*log10((1+z)*proper distance*H0/c)
    m(z) = A + B(Omegam, Omegalamb)
    input : 
        func : proper distance*H0/c (Other_stuff_flat or Other_stuff_curved)
        parm : [Omegam, Omegalamb] 
        z : redshift
    output :
        Bval : B(Omegam, Omegalamb)
    """
    funcval = func(z, parm) # proper distance*H0/c
    Bval = 5*np.log10((1+z)*funcval)
    return Bval

In [46]:
Omegam = np.linspace(0,20,50)
Omegal = np.linspace(-3,3,50)
M,L = np.meshgrid(Omegam, Omegal)
z = sndata['zcmb'].values.max()
A = B(Other_stuff_curved,[M,L],z)

/var/folders/h8/5hxv73fd07d033x49543ghl00000gn/T/ipykernel_8287/1102717268.py:4: RuntimeWarning: invalid value encountered in sqrt
  E = np.sqrt(E2)


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [41]:
print(M[np.where(np.isnan(A))])
print(L[np.where(np.isnan(A))])

[0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.40816327 0.         0.40816327
 0.         0.40816327 0.         0.40816327 0.         0.40816327
 0.         0.40816327 0.         0.40816327 0.         0.40816327
 0.81632653]
[1.16326531 1.28571429 1.40816327 1.53061224 1.65306122 1.7755102
 1.89795918 2.02040816 2.14285714 2.14285714 2.26530612 2.26530612
 2.3877551  2.3877551  2.51020408 2.51020408 2.63265306 2.63265306
 2.75510204 2.75510204 2.87755102 2.87755102 3.         3.
 3.        ]


In [43]:
np.isnan(np.log10(-1))

/var/folders/h8/5hxv73fd07d033x49543ghl00000gn/T/ipykernel_8287/3017672069.py:1: RuntimeWarning: invalid value encountered in log10
  np.isnan(np.log10(-1))


True